In [1]:
from nltk.tokenize import word_tokenize
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from csv import writer
import math
from gensim import  models
from gensim.test.utils import datapath

In [2]:
def load_books_chunks_from_document(CONST, file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    books = []
    books_info = []
    start_tag = '<doc title="'
    end_tag = '</doc>'
    start_index = 0

    while True:
        book_start = content.find(start_tag, start_index)
        if book_start == -1:
            break

        book_end = content.find(end_tag, book_start)
        if book_end == -1:
            break

        book_text = content[book_start:book_end + len(end_tag)]
        book_info = book_text.strip()[5:book_text.index('>') + 1]  # Remove '<doc' and '</doc>'

        book_info_list = book_info.split('" ')
        book_info_dict = {}

        for item in book_info_list:
            key, value = item.split('=')
            book_info_dict[key.strip()] = value.strip('"')

        book_content = book_text[book_text.index('>') + 1:-len(end_tag)].strip()
        book_content = book_content.split(' ')
        length = len(book_content)
        for i in range(math.ceil(length/CONST)):
            i = i*CONST
            end = i+CONST if i+CONST < length-1 else length-1
            books.append(" ".join(book_content[i:end]))
            books_info.append(book_info_dict)

        start_index = book_end + len(end_tag)

    return books, books_info


In [3]:
path = "data/lem"
year = "1990_2019"
category = "N_A"
file_name = year+"_"+category+"_no_names_beletrie"
file_path = path+"/"+file_name + ".txt"
chunk = 1000
divide = "chunk"

num_topics = 30


for year in ["1990_1999", "2000_2009", "2010_2019"]:
    for divide in ["chunk"]: 
            for chunk in [1000, 2000]:
                print("Year: {year}, chunk: {chunk}".format(year = year, chunk = chunk))

                file_name = year+"_"+category+"_no_names_beletrie"
                file_path = path+"/"+file_name + ".txt"
                books, books_info = load_books_chunks_from_document(chunk, file_path)
                d = str(chunk) + " " + divide
                save_path = "../../models/{divide}/lda/lda_{topics}_topics".format(divide = d, topics = num_topics) + file_name

                # Tokenize the documents
                tokenized_data = [word_tokenize(doc) for doc in books]

                # Create a dictionary from the preprocessed data
                dictionary = Dictionary(tokenized_data)

                # Create a document-term matrix
                doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_data]

                temp_file = datapath(save_path)

                lda_model = models.ldamodel.LdaModel.load(temp_file)

                #if __name__ == "__main__":
                for i in range(0,1):
                    # Calculate topic coherence using Gensim's CoherenceModel
                    coherence_model = CoherenceModel(
                        #topics=topic_words,
                        model = lda_model,
                        texts=tokenized_data,
                        corpus = doc_term_matrix,
                        dictionary=dictionary,
                        coherence='c_v'  # You can use other coherence measures as well
                        )
                    coherence_score = coherence_model.get_coherence()

                    print("Topic Coherence Score:", coherence_score)

                    List = [category, year, divide, chunk, num_topics ,coherence_score]
                    # Open our existing CSV file in append mode
                    # Create a file object for this file
                    with open('../../data/lda_results.csv', 'a') as f_object:
 
                    # Pass this file object to csv.writer()
                    # and get a writer object
                        writer_object = writer(f_object)
 
                        # Pass the list as an argument into
                        # the writerow()
                        writer_object.writerow(List)
 
                        # Close the file object
                        f_object.close()       

Year: 1990_1999, chunk: 1000
Topic Coherence Score: 0.4112844896419111
Year: 1990_1999, chunk: 2000
Topic Coherence Score: 0.41339156145351713
Year: 2000_2009, chunk: 1000
Topic Coherence Score: 0.3668567517932041
Year: 2000_2009, chunk: 2000
Topic Coherence Score: 0.35301361023959493
Year: 2010_2019, chunk: 1000
Topic Coherence Score: 0.4112370240553026
Year: 2010_2019, chunk: 2000
Topic Coherence Score: 0.40807349000715837
